In [ ]:
from rag_tests.db.mongo import connection
from rag_tests.core import get_logger
logger = get_logger(__name__)
mongo_db_connector = connection

In [ ]:
def fetch_posts_from_mongo_by_author_firstname(firstname:str):
    _database = mongo_db_connector.get_database("scrabble")
    collection = _database["posts"]
    return collection.find({"author_id": {"$regex": firstname, "$options": "i"}})

In [ ]:
from rag_tests.models.base import BaseModel
from rag_tests.models import PostsRawModel, PostChunkModel, PostCleanedModel, PostEmbeddedChunkModel
from qdrant_client.models import Batch
from rag_tests.models.clean import PostCleanedModel
from rag_tests.data_logic.cleaning_data_handlers import PostCleaningHandler
from rag_tests.data_logic.chunking_data_handlers import PostChunkingHandler
from rag_tests.data_logic.embedding_data_handlers import PostEmbeddingDataHandler
from rag_tests.qdrant_db import QdrantDatabaseConnector
from rag_tests.config import settings
collection_name = f"TEST_VARUN_{settings.EMBEDDING_MODEL_ID.replace('/','_')}"

In [ ]:
cursor = fetch_posts_from_mongo_by_author_firstname("varun")
raw_posts = []
seen_posts = set()
flag =True
for post in cursor:
    if flag : 
        print(post.keys())
        flag = False
    entry_id = post["_id"]
    if entry_id not in seen_posts:
        seen_posts.add(entry_id)
        # remove _id from post
        post.pop("_id")

        raw_posts.append(PostsRawModel(entry_id=entry_id, **post))


In [ ]:
# raw_posts= raw_posts[:110]

In [ ]:
cleaning_handler  = PostCleaningHandler()
cleaned_posts = []
for raw_post in raw_posts:
    cleaned_obj = cleaning_handler.clean(raw_post)
    cleaned_posts.append(cleaned_obj)

print(f"Number of cleaned posts: {len(cleaned_posts)}")


In [ ]:

chunking_handler = PostChunkingHandler()
chunked_posts = []
for cleaned_post in cleaned_posts:
    chunked_obj = chunking_handler.chunk(cleaned_post)
    chunked_posts.append(chunked_obj)

print(f"Number of chunked posts: {len(chunked_posts)}")


In [ ]:

embedding_handler = PostEmbeddingDataHandler()
embedded_posts:list[PostEmbeddedChunkModel] = []
for chunked_post in chunked_posts:
    embedded_obj = embedding_handler.embedd(chunked_post[0])
    embedded_posts.append(embedded_obj)

print(f"Number of embedded posts: {len(embedded_posts)}")


In [ ]:
qdrant_db = QdrantDatabaseConnector()
payloads = [
    post.to_payload() for post in embedded_posts
]
ids, vectors, metadatas = zip(*payloads)

collection_name = f"TEST_VARUN_{settings.EMBEDDING_MODEL_ID.replace('/','_')}"
qdrant_db.create_vector_collection(collection_name)
qdrant_db.write_data(collection_name, points=Batch(ids=ids, vectors=vectors, payloads=metadatas))
print(f"Chala bhai daaldi {len(embedded_posts)} posts qdrant mei")